In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import os
import zipfile
import sys

In [ ]:
if 'data' not in os.listdir():
    os.system("mkdir data && cd data && wget 'https://cloud.tsinghua.edu.cn/f/5f2421dd8a324c15b5ed/?dl=1' -O tabularData.csv")

In [29]:
tabularDf = pd.read_csv('data/tabularData.csv')

In [32]:
tabularDf = tabularDf[['open', 'high', 'low', 'close', 'volume', 'amount']]
tabularDf.to_csv('data/tabularDf.csv')

In [5]:
int(2009*1e4)

20090000

In [6]:
[year for year in range(2009, 2022, 2)]

[2009, 2011, 2013, 2015, 2017, 2019, 2021]

In [15]:
Training_ranges = [(int(start_year*1e4+1231-60), int(end_year*1e4+1231)) for start_year, end_year in zip(range(2009, 2019, 1), range(2011, 2021, 1))]
print(Training_ranges)

Test_ranges = [(int(start_year*1e4+1231-60), int(end_year*1e4+1231)) for start_year, end_year in zip(range(2011, 2021, 1), range(2012, 2022, 1))]
print(Test_ranges)

[(20091171, 20111231), (20101171, 20121231), (20111171, 20131231), (20121171, 20141231), (20131171, 20151231), (20141171, 20161231), (20151171, 20171231), (20161171, 20181231), (20171171, 20191231), (20181171, 20201231)]
[(20111171, 20121231), (20121171, 20131231), (20131171, 20141231), (20141171, 20151231), (20151171, 20161231), (20161171, 20171231), (20171171, 20181231), (20181171, 20191231), (20191171, 20201231), (20201171, 20211231)]


In [ ]:
Training_ranges = [(start_year*1e4+1231-60, end_year+1231), (20101231-60, 20121231), (20111231-60, 20131231), (20121231-60, 20141231), (20131231-60, 20151231), (20141231-60, 20161231), (20151231-60, 20171231), (20161231-60, 20181231), (20171231-60, 20191231), (20181231-60, 20201231)]

Test_ranges = [(20111231-60, 20121231), (20121231-60, 20131231), (20131231-60, 20141231), (20141231-60, 20151231), (20151231-60, 20161231), (20161231-60, 20171231), (20171231-60, 20181231), (20181231-60, 20191231), (20191231-60, 20201231), (20201231-60, 20211231)]

In [ ]:
tabularDf.head()

In [ ]:
tabularDf = tabularDf[['code', 'date', 'open', 'high', 'low', 'close', 'volume', 'amount']]
tabularDf['ret1'] = np.zeros(tabularDf.shape[0])
tabularDf['ret5'] = np.zeros(tabularDf.shape[0])
tabularDf['ret20'] = np.zeros(tabularDf.shape[0])

In [ ]:
symbol_data = []
for g in tqdm(tabularDf.groupby('code')):
    g[1]['ret1'] = (g[1]['close'].pct_change()*100).shift(-1)
    g[1]['ret5'] = (g[1]['close'].pct_change(5)*100).shift(-5)
    g[1]['ret20'] = (g[1]['close'].pct_change(20)*100).shift(-20)
    symbol_data.append(g[1])
    
tabularDf = pd.concat(symbol_data)

In [ ]:
os.system('cd data && mkdir Split_Data')
os.system('cd data/Split_Data && mkdir Train')
for idx, train_period in enumerate(Training_ranges[:3]):
    train_set = tabularDf.loc[(train_period[0] < tabularDf['date']) & (tabularDf['date'] < train_period[1])].reset_index(drop=True)
    train_set.to_csv('data/Split_Data/Train/train_set{:0>2}.csv'.format(idx+1))

os.system('cd data/Split_Data && mkdir Test')
for idx, test_period in enumerate(Test_ranges[:3]):
    test_set = tabularDf.loc[(test_period[0] < tabularDf['date']) & (tabularDf['date'] < test_period[1])].reset_index(drop=True)
    test_set.to_csv('data/Split_Data/Test/test_set{:0>2}.csv'.format(idx+1))

In [ ]:
def writeAllFileToZip(absDir, zipFile):
    for f in os.listdir(absDir):
        absFile=os.path.join(absDir,f) 
        if os.path.isdir(absFile): 
            relFile=absFile[len(os.getcwd())+1:] 
            zipFile.write(relFile) 
            writeAllFileToZip(absFile,zipFile) 
        else:
            relFile=absFile[len(os.getcwd())+1:]
            zipFile.write(relFile)
    return
    
zipFilePath=os.path.join(sys.path[0],"data/Split_Data.zip") 

zipFile=zipfile.ZipFile(zipFilePath, "w", zipfile.ZIP_DEFLATED) 
absDir=os.path.join(sys.path[0],"data/Split_Data/") 

writeAllFileToZip(absDir,zipFile) 
print("Successfully Compressed")


In [ ]:
class DataLoader():
    def __init__(self, win_size, start_date, end_date, batch_size, mode, shuffle):
        assert mode in ['train', 'test'], f'Type Error: {mode}'
        assert mode=='test' and not shuffle, 'Test Data should not shuffle'
        
        self.win_size = win_size # 5 or 15
        self.start_date =start_date
        self.end_date = end_date 
        self.batch_size = batch_size
        self.mode = mode
        self.shuffle = shuffle
        
                
    def generate_data():
        # generate data here, online or offline
        self.df = pd.DataFrame()
        
        
        
    def next_batch():
        # 
        
        if mode == 'train':
            
            return new_batch, labels # new_batch: (N, 32/64, win_size*3); labels: (N, 1)
        
        else: 
            
            
            # In test_mode: one stock, one batch
            # symbol: stock code; datelist: list of str; new_batch: (len(datelist), 32/64, win_size*3); labels: (len(datelist), 1)
            return symbol, datelist, new_batch, labels
    